In [1]:
# Import modules
from src.modules.data_loader import create_ffn_dataset
from src.models.fnn import build_ffn
#from models.ffn import build_ffn
#from modules.training_utils import train_ffn

2025-01-29 11:37:27.590894: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-29 11:37:27.629711: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-29 11:37:27.630611: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-29 11:37:28.298775: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
# File paths
tfrecord_file = "../data/FFNs/FFNs_init.tfrecord"

# Input parameters
max_atoms = 7662 # Required for fixed sized FNN
input_geometry_shape = (max_atoms, 3)  # Example geometry shape
input_frequency_shape = (1,)

# Training hyperparameters
batch_size = 32
epochs = 20
learning_rate = 1e-3

In [3]:
# Load dataset
print("Loading dataset...")
dataset = create_ffn_dataset(tfrecord_file, batch_size=batch_size)
print("Dataset loaded.")

Dataset loaded.


In [ ]:
# Build model
print("Building model...")
model = build_ffn(input_geometry_shape, input_frequency_shape)
model.summary()